In [1]:
import pandas as pd

In [4]:
def getData():
  data = pd.read_csv("LoanStats_2018Q3.csv", skiprows=[0])
  data.drop('member_id', axis=1, inplace=True)
  data.drop('id', axis=1, inplace=True)
# The last two rows are all missing values (only 43 are missing in the last third row), drop them. The last two rows are missing in the original excel dataset
  data.drop([128194,128195], axis=0, inplace=True)
  headers = list(data)
# find out features with only one unique level and drop them

# pymnt_plan has only one level, which is 'n', and has no missing values
  headers_unique = list(data.columns[data.nunique()==1])
  for header in headers_unique:
      print(header)
      print(data[header].unique())
      print(data[header].isna().sum())
      print()
# 128181 people have no hardship records becasue there are 128181 missing values in hardship-related features. It seems that 13 people experienced hardship and have completed it, but it is still an intuitively valuble predictor for indicating a person would likely default, so keep them and drop other unique level features first
  for header in headers_unique[:-6]:
      data.drop(header, axis=1, inplace=True)
# Since all hardship features are indicating the same thing (3 months length of hardship term), only need to keep one hardship feature (the same for hardship_reason becuase it also only has one level). Keep hardship_status as a categorical feature and name missing values by 'No' as another category
  for header in headers_unique[-6:-3]:
      data.drop(header, axis=1, inplace=True)
  for header in headers_unique[-2:]:
      data.drop(header, axis=1, inplace=True)
  data['hardship_status'].fillna('No',inplace=True)
  headers_na = list(data.columns[data.isna().sum() > 0])
# impute hardship-related features for first
  headers_na_hard = ['hardship_amount', 'hardship_start_date', 'hardship_end_date', 'payment_plan_start_date', 'hardship_dpd',
                   'hardship_loan_status', 'orig_projected_additional_accrued_interest', 'hardship_payoff_balance_amount',
                   'hardship_last_payment_amount' ]

  #哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈哈
  for header in headers_na_hard:
      print(header)
      print(data[header].unique())
      print(data[header].isna().sum())
      print()

  print(data['hardship_status'].unique())
  print(data['hardship_status'].isna().sum())

  data.drop(labels=['hardship_start_date','hardship_end_date','payment_plan_start_date',
                              'hardship_dpd'],axis=1,inplace=True)


# impute missing values in hardship_amount, orig_projected_additional_accrued_interest, hardship_payoff_balance_amount, and hardship_last_payment_amount by 0 (float), and name missing values in hardship_loan_status by 'No' as a new category
  data['hardship_loan_status'].fillna('No',inplace=True)
  data['hardship_amount'].fillna(0.,inplace=True)
  data['orig_projected_additional_accrued_interest'].fillna(0.,inplace=True)
  data['hardship_payoff_balance_amount'].fillna(0.,inplace=True)
  data['hardship_last_payment_amount'].fillna(0.,inplace=True)

  data.groupby(by=['hardship_loan_status']).size()
# then, impute settlement-related features
  headers_na_settle = ['debt_settlement_flag_date', 'settlement_status', 'settlement_date', 'settlement_amount',
                     'settlement_percentage', 'settlement_term']

  for header in headers_na_settle:
      print(header)
      print(data[header].unique())
      print(data[header].isna().sum())
      print()

  data.groupby(by=['settlement_date']).size()
  data.groupby(by=['debt_settlement_flag_date']).size()
  data.groupby(by=['settlement_percentage']).size()
  data.groupby(by=['settlement_status']).size()
  settle_index = list(data.index[data['debt_settlement_flag_date'].isna()==False])
  charged_off_index = list(data.index[data['loan_status'] == 'Charged Off'])

# Not all people on settlement plan are marked as charged off in loan_status 
  settle_charge=[]
  settle_not_in_charge=[]
  for i in settle_index:
      if i in charged_off_index:
          settle_charge.append(i)
      else:
          settle_not_in_charge.append(i)
  print(settle_charge)
  print(settle_not_in_charge)

  head = ['settlement_status', 'loan_status']

  for i in settle_not_in_charge:
      print(data[head][i:i+1])

# people on settlement plan are also in other loan status besides being charged off
  data.groupby(by=['debt_settlement_flag']).size()


# drop settlement date related features
  data.drop(labels=['debt_settlement_flag_date','settlement_date'],axis=1,inplace=True)
  data['settlement_status'].fillna('No',inplace=True)
  data['settlement_amount'].fillna(0.,inplace=True)
  data['settlement_percentage'].fillna(0.,inplace=True)
  data['settlement_term'].fillna(0.,inplace=True)

# inspect remaining features that have missing values
  headers_na_rest = list(data.columns[data.isna().sum() > 0])
  print(len(headers_na_rest),'remaining features to be imputed')
  data[headers_na_rest].isna().sum()

# impute features that are related to secondary or joint applicant

  headers_na_sec = ['sec_app_earliest_cr_line', 'sec_app_inq_last_6mths','sec_app_mort_acc','sec_app_open_acc','sec_app_revol_util',
                 'sec_app_open_act_il','sec_app_num_rev_accts','sec_app_chargeoff_within_12_mths','sec_app_collections_12_mths_ex_med',
                 'sec_app_mths_since_last_major_derog','revol_bal_joint','verification_status_joint','dti_joint','annual_inc_joint']

  for header in headers_na_sec:
      print(header)
      print(data[header].unique())
      print()

  data[headers_na_sec].isna().sum()
# sec_app_revol_util, sec_app_mths_since_last_major_derog, and verification_status_joint have different number of NA than others. sec_app_mths_since_last_major_derog is ok becuase it may not apply to some secondary applicants. examine the other two features
  sec_index_base1 = list(data.index[data['sec_app_earliest_cr_line'].isna()==False])
  sec_index_base2 = list(data.index[data['sec_app_mort_acc'].isna()==False])

  all(i in sec_index_base1 for i in sec_index_base2)
# confirm indexes of features that have 110409 NAs matching first, and use the Non-NAs' indexes as baseline for comparison; confirm if all indexes of Non-NAs in sec_app_revol_util and verification_status_joint are in baseline
  sec_index_revol = list(data.index[data['sec_app_revol_util'].isna()==False])
  sec_index_veri = list(data.index[data['verification_status_joint'].isna()==False])

  sec_diff_revol=[]
  sec_diff_veri=[]
  for i in sec_index_base1:
      if i not in sec_index_revol:
          sec_diff_revol.append(i)

  for i in sec_index_base1:
      if i not in sec_index_veri:
          sec_diff_veri.append(i)

# confirmed all indexes in sec_app_revol_util and verification_status_joint are in baseline
  data.groupby(by=['verification_status_joint']).size()
  any(data['sec_app_revol_util']==0.)

# Since there are data=0 in sec_app_revol_util, cannot use 0 to impute missing values of people not having secondary applicant, use -1 instead. Use median to impute missing values of people having secondary applicant. 
# Use 'Not Verified' to impute verification_status_joint for people having secondary applicant, and 'No' to impute people not having secondary applicant

  for i in sec_diff_revol:
      data['sec_app_revol_util'][i:i+1].fillna(data['sec_app_revol_util'].median(),inplace=True)

  data['sec_app_revol_util'].fillna(-1.,inplace=True)

  for i in sec_diff_veri:
      data['verification_status_joint'][i:i+1].fillna('Not Verified',inplace=True)

  data['verification_status_joint'].fillna('No',inplace=True)
  data.groupby(by=['verification_status_joint']).size()
  data['sec_app_mths_since_last_major_derog'].fillna(-1.,inplace=True)
# now impute the rest secondary applicant related features by -1 because they are all numeric predictors, except for sec_app_earliest_cr_line because it is a date type predictor
  for header in headers_na_sec:
      if header not in ['sec_app_revol_util','verification_status_joint','sec_app_mths_since_last_major_derog', 'sec_app_earliest_cr_line']:
          data[header].fillna(-1.,inplace=True)

  data['sec_app_earliest_cr_line'].dtype

# now deal with date type predictor sec_app_earliest_cr_line; impute it with another category, say 'Jan-1900'
  data['sec_app_earliest_cr_line'].fillna('Jan-1900',inplace=True)


# Again, inspect remaining features that have missing values
  headers_na_rest1 = list(data.columns[data.isna().sum() > 0])
  print(len(headers_na_rest1),'remaining features to be imputed')
  data[headers_na_rest1].isna().sum()


# drop url and desc becuase the two columns are entirely missing
  data.drop(labels=['url','desc'], axis=1, inplace=True)

# drop emp_title because of too many levels
  data.drop('emp_title', axis=1,inplace=True)
# inspect features that have less than 1000 missing values, impute numeric predictor by mean or median, categorical by mode
  data['dti'].dtype
  data['dti'].fillna(data['dti'].mean(),inplace=True)
  data['revol_util'] = data['revol_util'].str.rstrip('%').astype('float')
  data['revol_util'].fillna(data['revol_util'].mean(),inplace=True)
  data.groupby(by=['last_pymnt_d']).size()
  data['last_pymnt_d'].fillna('Feb-2019',inplace=True)
  data.groupby(by=['last_credit_pull_d']).size()
  data['last_credit_pull_d'].fillna('Feb-2019',inplace=True)
  data['all_util'].fillna(data['all_util'].mean(),inplace=True)
  data['avg_cur_bal'].fillna(data['avg_cur_bal'].median(),inplace=True)
  data['pct_tl_nvr_dlq'].fillna(data['pct_tl_nvr_dlq'].mean(),inplace=True)

# inspect features that have more than 1000 missing values; use other features to predict missing values
  headers_na_rest2 = list(data.columns[data.isna().sum() > 0])
  print(len(headers_na_rest2),'remaining features to be imputed')
  data[headers_na_rest2].isna().sum()
  for header in headers_na_rest2:
      print(header)
      print(data[header].dtype)
      print()
  data.groupby(by=['loan_status']).size()

  data['term'] = data['term'].str.strip()
  data['term'] = data['term'].str.rstrip(' months')

  # for column in data.columns:
  #   if data[column].dtype == "object":
  #     data[column] = data[column].str.strip()
  #     data[column] = data[column].str.strip('%')

#   data.to_csv('PartitiallyImputedData.csv')
  return data
#   headersLast = list(data)
#   for i in headersLast:
#     if data[i].dtype == 'O':
#       data[i] = data[i].astype('category')
#       data[i] = data[i].cat.codes

#   print(data.dtypes)
#   data.to_csv('miceData_category.csv')

In [5]:
data

NameError: name 'data' is not defined

In [8]:
def categorical():
    data = getData()
    headersLast = list(data)
    for i in headersLast:
        if data[i].dtype == 'O':
          data[i] = data[i].astype('category')
        
#           data[i] = data[i].cat.codes

    print(data.dtypes)
#     data.to_csv('miceData_category.csv')

In [9]:
categorical()

/anaconda/envs/py35/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2963: DtypeWarning: Columns (0,123,124,125,128,129,130,133,139,140,141) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


pymnt_plan
['n']
0

policy_code
[1.]
0

acc_now_delinq
[0.]
0

num_tl_120dpd_2m
[ 0. nan]
3254

num_tl_30dpd
[0.]
0

tax_liens
[0.]
0

hardship_flag
['N']
0

hardship_type
[nan 'INTEREST ONLY-3 MONTHS DEFERRAL']
128181

hardship_reason
[nan 'NATURAL_DISASTER']
128181

hardship_status
[nan 'COMPLETED']
128181

deferral_term
[nan  3.]
128181

hardship_length
[nan  3.]
128181

hardship_amount
[   nan 203.67 378.9  126.27 119.59 243.56 159.94 241.9  134.81 194.82
  55.04 215.7  248.46 109.26]
128181

hardship_start_date
[nan 'Oct-2018' 'Nov-2018' 'Sep-2018']
128181

hardship_end_date
[nan 'Dec-2018' 'Jan-2019' 'Feb-2019']
128181

payment_plan_start_date
[nan 'Oct-2018' 'Nov-2018' 'Dec-2018']
128181

hardship_dpd
[nan  0. 18.]
128181

hardship_loan_status
[nan 'Issued' 'Current' 'Late (31-120 days)']
128181

orig_projected_additional_accrued_interest
[    nan  611.01 1136.7   378.81  358.77  730.68  479.82  725.7   404.43
  584.46  165.12  647.1   745.38  327.78]
128181

hardship_payoff_bal

25 remaining features to be imputed
15 remaining features to be imputed
emp_length
object

mths_since_last_delinq
float64

mths_since_last_record
float64

next_pymnt_d
object

mths_since_last_major_derog
float64

mths_since_rcnt_il
float64

il_util
float64

bc_open_to_buy
float64

bc_util
float64

mo_sin_old_il_acct
float64

mths_since_recent_bc
float64

mths_since_recent_bc_dlq
float64

mths_since_recent_inq
float64

mths_since_recent_revol_delinq
float64

percent_bc_gt_75
float64

loan_amnt                                      float64
funded_amnt                                    float64
funded_amnt_inv                                float64
term                                          category
int_rate                                      category
installment                                    float64
grade                                         category
sub_grade                                     category
emp_length                                    category
home_ownership   